In [1]:
import pandas as pd
print(pd.__version__)
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

0.23.4


## Standard Functions

In [2]:
def calcPer(df_input):
    print("Number of Positives", sum(df_input[inference]==1))
    print("Total Number of Examples:", len(df_input))
    print("Percentage of positives:", int(100*(sum(df_input[inference]==1)/len(df_input))), "%")
def undersample(input_df, output, numNegatives):
    new_df = input_df[input_df[output]==0]
    new_df = shuffle(new_df)[:numNegatives]
    return shuffle(pd.concat([new_df, input_df[input_df[output]==1]]))

In [3]:
standardized = ['rel_app', 'era', 'hits9', 'hr9', 'bb9', 'so9', 'so_bb', 'whip', 
                'gs_pct', 'ip_gm', 'hgt_inches', 'wgt', 'wins', 'gs', 'gm', 
                'sv', 'gf', 'sho']
binaries = ['mlb_binary', 'ba100_post']
inference = 'org_top10_post'
all_features = standardized + binaries + [inference]
sub = all_features + ['year']
print(all_features)

['rel_app', 'era', 'hits9', 'hr9', 'bb9', 'so9', 'so_bb', 'whip', 'gs_pct', 'ip_gm', 'hgt_inches', 'wgt', 'wins', 'gs', 'gm', 'sv', 'gf', 'sho', 'mlb_binary', 'ba100_post', 'org_top10_post']


In [4]:
df = pd.read_csv("pitch_project_data.csv")
df = df[sub]
df.dropna(inplace=True)
calcPer(df)
df.head()

Number of Positives 4188
Total Number of Examples: 95023
Percentage of positives: 4 %


,rel_app,era,hits9,hr9,bb9,so9,so_bb,whip,gs_pct,ip_gm,...,wins,gs,gm,sv,gf,sho,mlb_binary,ba100_post,org_top10_post,year
0,2,2.386740,6.414365,0.149171,0.895028,8.204420,9.166667,0.812155,0.866667,4.022222,...,5,13,15,0.0,0.0,0,1,0,0,2000
1,0,3.681818,8.744318,1.073864,1.380682,6.903409,5.000000,1.125000,1.000000,6.518519,...,5,9,9,0.0,0.0,0,1,0,0,2001
2,0,3.687805,8.956098,0.921951,1.317073,6.980488,5.300000,1.141463,1.000000,6.212121,...,5,11,11,0.0,0.0,1,1,0,0,2001
3,0,4.285714,9.000000,0.857143,2.142857,5.000000,2.333333,1.238095,1.000000,5.727273,...,5,11,11,0.0,0.0,0,1,0,0,2001
4,0,5.114537,11.775330,0.475771,2.259912,4.162996,1.842105,1.559471,1.000000,5.404762,...,4,14,14,0.0,0.0,0,1,0,0,2002


In [5]:
testdf = df[df.year==2018]
trainvaldf = df[df.year!=2018]
alldf = df[df.year!=2018]
positivedf = df[df.year!=2018]
positivedf = positivedf[positivedf[inference]==1]
ndf = df[df.year!=2018]
ndf = ndf[ndf[inference]==0]
trainvaldf = undersample(trainvaldf, inference, 5000)
calcPer(trainvaldf)

Number of Positives 4188
Total Number of Examples: 9188
Percentage of positives: 45 %


In [6]:
num_standard = len(standardized); print("Standardized Features: ", num_standard)
num_binary = len(binaries)
train_s = trainvaldf[standardized]
test_s = testdf[standardized]
all_s = alldf[standardized]
positive_s = positivedf[standardized]
n_s = ndf[standardized]

scalar = StandardScaler().fit(all_s)
scaled_train_s = scalar.transform(train_s)
scaled_test_s = scalar.transform(test_s)
scaled_all_s = scalar.transform(all_s)
scaled_positive_s = scalar.transform(positive_s)
scaled_n_s = scalar.transform(n_s)
print("train standardized", scaled_train_s.shape); 
print("test standardized shape", scaled_test_s.shape)
print("all standardized shape", scaled_all_s.shape)
print("all standardized shape", scaled_positive_s.shape)


Standardized Features:  18
train standardized (9188, 18)
test standardized shape (7370, 18)
all standardized shape (87653, 18)
all standardized shape (4188, 18)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  # This is added back by InteractiveShellApp.init_path()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardSca

In [7]:
train_bin = trainvaldf[binaries]
test_bin = testdf[binaries]
all_bin = alldf[binaries]
positive_bin = positivedf[binaries]
n_bin = ndf[binaries]

train_preprocess = np.concatenate((scaled_train_s, train_bin), axis=1)
train_shape = train_preprocess.shape
test_preprocess = np.concatenate((scaled_test_s, test_bin), axis=1)
all_preprocess = np.concatenate((scaled_all_s, all_bin), axis=1)
positive_preprocess = np.concatenate((scaled_positive_s, positive_bin), axis=1)
n_preprocess = np.concatenate((scaled_n_s, n_bin), axis=1)

train_real = train_preprocess[:int(train_shape[0]*.9),:]
val_real = train_preprocess[int(train_shape[0]*.9):,:]
test_real = test_preprocess
all_real = all_preprocess
positive_real = positive_preprocess
n_real = n_preprocess

print("train standardized shape", train_real.shape); 
print("val standardized shape", val_real.shape)
print("test standardized shape", test_real.shape)
print("all standardized shape", all_real.shape)
print("positive standardized shape", positive_real.shape)

Y_train = np.array(list(trainvaldf[inference]))[:int(train_shape[0]*.9)]
Y_val = np.array(list(trainvaldf[inference]))[int(train_shape[0]*.9):]
Y_all = np.array(list(alldf[inference]))
Y_positive = np.array(list(positivedf[inference]))
Y_n = np.array(list(ndf[inference]))

print("train", Y_train.shape)
print("val", Y_val.shape)
print("test", Y_test.shape)
print("all", Y_all.shape)
print("positive", Y_positive.shape)

train standardized shape (8269, 20)
val standardized shape (919, 20)
test standardized shape (7370, 20)
all standardized shape (87653, 20)
positive standardized shape (4188, 20)
train (8269,)
val (919,)


NameError: name 'Y_test' is not defined

In [8]:
print("Evaluate NA Values")
numNAValues = 0
for col in df.columns:
    if sum(pd.isna(df[col])) > 0:
        print(col, sum(pd.isna(df[col])))
        numNAValues += 1
if numNAValues == 0:
    print("No NA Values")
else:
    print("Number of Columns with NA Values:", numNAValues)

Evaluate NA Values
No NA Values


In [9]:
print("Positive Y_vals", sum(Y_val))
print("Positive All_vals", sum(Y_all))

Positive Y_vals 433
Positive All_vals 4188


In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "SVC Gamma",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    SVC(gamma='auto'),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

# iterate over classifiers
for name, clf in zip(names, classifiers):
    print("Begin Training on:", name)
    clf.fit(train_real, Y_train)
    print("Score on classifier:", name)
    print("validation:", clf.score(val_real, Y_val))
    print("sensitivity:", clf.score(positive_real, Y_positive))
    print("specifity:", clf.score(n_real, Y_n))
    print("all:", clf.score(all_real, Y_all))
    print("------")
    print()


Begin Training on: Nearest Neighbors
Score on classifier: Nearest Neighbors
validation: 0.7889009793253536
sensitivity: 0.8467048710601719
specifity: 0.8116336188821662
all: 0.8133092991683114
------

Begin Training on: Linear SVM
Score on classifier: Linear SVM
validation: 0.7986942328618063
sensitivity: 0.7480897803247374
specifity: 0.844246091175942
all: 0.8396518088371191
------

Begin Training on: RBF SVM
Score on classifier: RBF SVM
validation: 0.6866158868335147
sensitivity: 0.934574976122254
specifity: 0.9398071047744564
all: 0.9395571172692321
------

Begin Training on: SVC Gamma
Score on classifier: SVC Gamma
validation: 0.8248095756256801
sensitivity: 0.7870105062082139
specifity: 0.8561672557359372
all: 0.8528629938507524
------

Begin Training on: Decision Tree
Score on classifier: Decision Tree
validation: 0.795429815016322
sensitivity: 0.7612225405921681
specifity: 0.8530042532798179
all: 0.8486189862297925
------

Begin Training on: Random Forest
Score on classifier: Ra

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:716: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:716: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:719: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:716: RuntimeWarning: divide by zero encountered in po

In [12]:
clf = SVC(gamma='auto')
clf.fit(train_real, Y_train)
ans = clf.predict(test_real)
print(ans)

[0 0 0 ... 0 0 0]


In [13]:
sum(ans)/len(ans)

0.1405698778833107

In [14]:
fin = np.nonzero(ans)[0].tolist()

In [17]:
df = pd.read_csv("pitch_project_data.csv")
df = df[sub + ['firstName', 'lastName', 'bcubeID']]
df.dropna(inplace=True)
testdf = df[df.year==2018]
results = testdf.iloc[fin,:]
results = results[['firstName', 'lastName', 'bcubeID']]
results

,firstName,lastName,bcubeID
963,Christian,Bergman,1810
4504,Jumbo,Diaz,5130
6423,Edwin,Jackson,5755
6424,Edwin,Jackson,5755
6425,Edwin,Jackson,5755
7189,Francisco,Liriano,6000
7511,J.D.,Martin,6128
11242,Adam,Wainwright,7368
11243,Adam,Wainwright,7368
11244,Adam,Wainwright,7368


In [18]:
results.to_csv("results.csv", index=False, header=True)
print("File Saved to ", "results.csv")

File Saved to  results.csv
